In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import torch
import h5py
from ipywidgets import interact

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from torch import nn
import clip
from collections import OrderedDict

clip_model, _ = clip.load('RN50x16', device='cpu')
clip_model = nn.Sequential(OrderedDict([
    ('wrapped', clip_model.visual),
    ('logits', nn.Identity())
]))

In [ ]:
clip_module.named_modules()

In [6]:
from pathlib import Path
from collections import OrderedDict

from model_tools.check_submission import check_models
import torch
from torch import nn
import functools
from model_tools.activations.pytorch import PytorchWrapper
from model_tools.brain_transformation import ModelCommitment
from model_tools.activations.pytorch import load_preprocess_images
from brainscore import score_model

import clip

model_definitions = {
    'RN50': {'layers': ['layer2.3.bn3', 'layer3.5.bn3', 'layer4.2.bn3', 'attnpool']},
    'RN101': {'layers': ['layer2.3.bn3', 'layer3.22.bn3', 'layer4.2.bn3', 'attnpool']},
    'RN50x4': {'layers': ['layer2.5.bn3', 'layer3.9.bn3', 'layer4.5.bn3', 'attnpool']},
    'RN50x16': {'layers': ['layer2.7.bn3', 'layer3.17.bn3', 'layer4.7.bn3', 'attnpool']},
    'ViT-B/32': {'layers': [*[f'transformer.resblocks.{i}' for i in range(12)], 'ln_post']},
    'ViT-B/16': {'layers': [*[f'transformer.resblocks.{i}' for i in range(12)], 'ln_post']},
}

available_models = clip.available_models()
for model_name in available_models:
    if model_name not in model_definitions:
        print(f"Available clip model available which is not defined: {model_name}")


for model_name, model_definition in model_definitions.items():
    if model_name not in available_models:
        print(f"Clip model {model_name} was defined but is not available, skipping it")
        continue

    clip_model, preprocess = clip.load(model_name, device='cpu')

    file_path = Path('.') / 'zeroshot-weights' / f'{model_name.replace("/", "=")}-zeroshot-weights.pt'
    if not file_path.exists():
        print(f"Zeroshot weights not found for {model_name}, generating them.")
        make_clip_zeroshot_weights(model_name)
    zeroshot_weights = torch.load(file_path)

    class ZeroShotWeights(nn.Module):
        def __init__(self, zeroshot_weights: torch.Tensor):
            super().__init__()
            self.zeroshot_weights = zeroshot_weights

        def forward(self, x):
            x = x / x.norm(dim=-1, keepdim=True)
            logits = 100. * x @ self.zeroshot_weights.to(x.dtype).to(x.device)
            return logits

    clip_model = nn.Sequential(OrderedDict([
        ('wrapped', clip_model.visual),
        ('logits', ZeroShotWeights(zeroshot_weights))
    ]))

    resize = preprocess.transforms[0]
    normalize = preprocess.transforms[4]
    preprocessing = functools.partial(load_preprocess_images,
                                      image_size=resize.size,
                                      normalize_mean=normalize.mean,
                                      normalize_std=normalize.std)

    model_definition['layers'] = [f'wrapped.{layer}' for layer in model_definition['layers']]
    model_definition['activations'] = PytorchWrapper(identifier=model_name, model=clip_model,
                                                     preprocessing=preprocessing, batch_size=8)
    model_definition['model'] = ModelCommitment(identifier=model_name,
                                                activations_model=model_definition['activations'],
                                                layers=model_definition['layers'],)
                                                #behavioral_readout_layer=model_definition['layers'][-1])

In [4]:
from brainscore.benchmarks import public_benchmark_pool

public_benchmark_pool

Loading lookup from entrypoints
Loading lookup from C:\Users\Cefir\AppData\Roaming\Python\Python38\site-packages\brainscore\lookup.csv


{'movshon.FreemanZiemba2013public.V1-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x1a7fdbdea00>,
 'movshon.FreemanZiemba2013public.V2-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x1a7fdbdea90>,
 'dicarlo.MajajHong2015public.V4-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x1a7fdbded00>,
 'dicarlo.MajajHong2015public.IT-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x1a7fdbe7250>,
 'dicarlo.Rajalingham2018public-i2n': <brainscore.benchmarks.public_benchmarks.RajalinghamMatchtosamplePublicBenchmark at 0x1a7fdbe7460>,
 'fei-fei.Deng2009-top1': <brainscore.utils.LazyLoad at 0x1a7fdb65d30>,
 'dietterich.Hendrycks2019-noise-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x1a7fdb65fd0>,
 'dietterich.Hendrycks2019-blur-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x1a7fdb77190>,
 'dietterich.Hendrycks2019-weather-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x1a7fdbb1

In [7]:
from brainscore.benchmarks import public_benchmark_pool

benchmark = public_benchmark_pool['dicarlo.MajajHong2015public.IT-pls']

score = benchmark(model_definitions['RN50']['model'])

cross-validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [1:18:51<00:00, 473.17s/it]


In [15]:
import pickle

with open('test.pkl', 'wb') as f:
    pickle.dump(score, f)

In [16]:
with open('test.pkl', 'rb') as f:
    score_loaded = pickle.load(f)
print(score_loaded)

<xarray.Score (aggregation: 2)>
array([0.53534334, 0.00350558])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:      <xarray.Score (aggregation: 2)>\narray([0.59689724, 0.00350558]...
    ceiling:  <xarray.Score (aggregation: 2)>\narray([0.81579938, 0.00144955]...
